<a id='section1'></a>
<div style="
    width: 100%;
    height: 100px;
    border:none;
    background:linear-gradient(-45deg, #d84227, #ce3587);
    text-align: center;
    position: relative;
">
  <span style="
      font-size: 40px;
      padding: 0 10px;
      color: white;
      margin: 0;
      position: absolute;
      top: 50%;
      left: 50%;
      -ms-transform: translate(-50%, -50%);
      transform: translate(-50%, -50%);
      font-weight: bold;
  ">
    XBRL PARSER <!--Padding is optional-->
  </span>
</div><a id='section1'></a>

In [1]:
"""
Martin Wood - Office for National Statistics
martin.wood@ons.gov.uk
23/07/2018
XBRL parser
Contains functions that scrape and clean an XBRL document's content
and variables, returning a dict ready for dumping into MongoDB.

ELliott PHillips - Office for National Statistics
eelliott.phillips@ons.gov.uk
30/06/2020
Code adapted and furthered for Companies House XBRL accounts parsing and outputting.
"""

import os
import re

import numpy as np
import pandas as pd

from datetime import datetime
from dateutil import parser
from bs4 import BeautifulSoup as BS  # Can parse xml or html docs

# Table of variables and values that indicate consolidated status
consolidation_var_table = {
    "includedinconsolidationsubsidiary": True,
    "investmententityrequiredtoapplyexceptionfromconsolidationtruefalse": True,
    "subsidiaryunconsolidatedtruefalse": False,
    "descriptionreasonwhyentityhasnotpreparedconsolidatedfinancialstatements": "exist",
    "consolidationpolicy": "exist"
}


def clean_value(string):
    """
    Take a value that's stored as a string,
    clean it and convert to numeric.

    If it's just a dash, it's taken to mean
    zero.
    """
    if string.strip() == "-":
        return (0.0)

    try:
        return float(string.strip().replace(",", "").replace(" ", ""))
    except:
        pass

    return (string)


def retrieve_from_context(soup, contextref):
    """
    Used where an element of the document contained no data, only a
    reference to a context element.
    Finds the relevant context element and retrieves the relevant data.

    Returns a text string

    Keyword arguments:
    soup -- BeautifulSoup souped html/xml object
    contextref -- the id of the context element to be raided
    """
    try:
        context = soup.find("xbrli:context", id=contextref)
        contents = context.find("xbrldi:explicitmember").get_text().split(":")[-1].strip()

    except:
        contents = ""

    return (contents)


def retrieve_accounting_standard(soup):
    """
    Gets the account reporting standard in use in a document by hunting
    down the link to the schema reference sheet that always appears to
    be in the document, and extracting the format and standard date from
    the string of the url itself.
    WARNING - That means that there's a lot of implicity hardcoded info
    on the way these links are formated and referenced, within this
    function.  Might need changing someday.

    Returns a 3-tuple (standard, date, original url)

    Keyword arguments:
    soup -- BeautifulSoup souped html/xml object
    """

    # Find the relevant link by its unique attribute
    link_obj = soup.find("link:schemaref")

    # If we didn't find anything it's an xml doc using a different
    # element name:
    if link_obj == None:
        link_obj = soup.find("schemaref")

    # extract the name of the .xsd schema file, which contains format
    # and date information
    text = link_obj['xlink:href'].split("/")[-1].split(".")[0]

    # Split the extracted text into format and date, return values
    return (text[:-10].strip("-"), text[-10:], link_obj['xlink:href'])


def retrieve_unit(soup, each):
    """
    Gets the reporting unit by trying to chase a unitref to
    its source, alternatively uses element attribute unitref
    if it's not a reference to another element.

    Returns the unit

    Keyword arguments:
    soup -- BeautifulSoup souped html/xml object
    each -- element of BeautifulSoup souped object
    """

    # If not, try to discover the unit string in the
    # soup object
    try:
        unit_str = soup.find(id=each['unitref']).get_text()

    except:
        # Or if not, in the attributes of the element
        try:
            unit_str = each.attrs['unitref']

        except:
            return ("NA")

    return (unit_str.strip())


def retrieve_date(soup, each):
    """
    Gets the reporting date by trying to chase a contextref
    to its source and extract its period, alternatively uses
    element attribute contextref if it's not a reference
    to another element.
    Returns the date

    Keyword arguments:
    soup -- BeautifulSoup souped html/xml object
    each -- element of BeautifulSoup souped object
    """

    # Try to find a date tag within the contextref element,
    # starting with the most specific tags, and starting with
    # those for ixbrl docs as it's the most common file.
    date_tag_list = ["xbrli:enddate",
                     "xbrli:instant",
                     "xbrli:period",
                     "enddate",
                     "instant",
                     "period"]

    for tag in date_tag_list:
        try:
            date_str = each['contextref']
            date_val = parser.parse(soup.find(id=each['contextref']).find(tag).get_text()). \
                date(). \
                isoformat()
            return (date_val)
        except:
            pass

    try:
        date_str = each.attrs['contextref']
        date_val = parser.parse(each.attrs['contextref']). \
            date(). \
            isoformat()
        return (date_val)
    except:
        pass

    return ("NA")


def parse_element(soup, element):
    """
    For a discovered XBRL tagged element, go through, retrieve its name
    and value and associated metadata.

    Keyword arguments:
    soup -- BeautifulSoup object of accounts document
    element -- soup object of discovered tagged element
    """
    
    if "contextref" not in element.attrs:
        return ({})

    element_dict = {}

    # Basic name and value
    try:
        # Method for XBRLi docs first
        element_dict['name'] = element.attrs['name'].lower().split(":")[-1]
    except:
        # Method for XBRL docs second
        element_dict['name'] = element.name.lower().split(":")[-1]

    element_dict['value'] = element.get_text()
    element_dict['unit'] = retrieve_unit(soup, element)
    element_dict['date'] = retrieve_date(soup, element)

    # If there's no value retrieved, try raiding the associated context data
    if element_dict['value'] == "":
        element_dict['value'] = retrieve_from_context(soup, element.attrs['contextref'])

    # If the value has a defined unit (eg a currency) convert to numeric	
    if element_dict['unit'] != "NA":
        element_dict['value'] = clean_value(element_dict['value'])

    # Retrieve sign of element if exists
    try:
        element_dict['sign'] = element.attrs['sign']

        # if it's negative, convert the value then and there
        if element_dict['sign'].strip() == "-":
            element_dict['value'] = 0.0 - element_dict['value']
    except:
        pass

    return (element_dict)


def parse_elements(element_set, soup):
    """
    For a set of discovered elements within a document, try to parse
    them.  Only keep valid results (test is whether field "name"
    exists).

    Keyword arguments:
    element_set -- BeautifulSoup iterable search result object
    soup -- BeautifulSoup object of accounts document
    """
    elements = []
    for each in element_set:
        element_dict = parse_element(soup, each)
        if 'name' in element_dict:
            elements.append(element_dict)
    return (elements)


def summarise_by_sum(doc, variable_names):
    """
    Takes a document (dict) after extraction, and tries to extract
    a summary variable relating to the financial state of the enterprise
    by summing all those named that exist.  Returns dict.

    Keyword arguments:
    doc -- an extracted document dict, with "elements" entry as created
           by the 'scrape_clean_elements' functions.
    variable_names - variables to find and sum if they exist
    """

    # Convert elements to pandas df
    df = pd.DataFrame(doc['elements'])

    # Subset to most recent (latest dated)
    df = df[df['date'] == doc['doc_balancesheetdate']]

    total_assets = 0.0
    unit = "NA"

    # Find the total assets by summing components
    for each in variable_names:

        # Fault-tolerant, will skip whatever isn't numeric
        try:
            total_assets = total_assets + df[df['name'] == each].iloc[0]['value']

            # Retrieve reporting unit if exists
            unit = df[df['name'] == each].iloc[0]['unit']

        except:
            pass

    return ({"total_assets": total_assets, "unit": unit})


def summarise_by_priority(doc, variable_names):
    """
    Takes a document (dict) after extraction, and tries to extract
    a summary variable relating to the financial state of the enterprise
    by looking for each named, in order.  Returns dict.

    Keyword arguments:
    doc -- an extracted document dict, with "elements" entry as created
           by the 'scrape_clean_elements' functions.
    variable_names - variables to find and check if they exist.
    """

    # Convert elements to pandas df
    df = pd.DataFrame(doc['elements'])

    # Subset to most recent (latest dated)
    df = df[df['date'] == doc['doc_balancesheetdate']]

    primary_assets = 0.0
    unit = "NA"

    # Find the net asset/liability variable by hunting names in order
    for each in variable_names:
        try:

            # Fault tolerant, will skip whatever isn't numeric
            primary_assets = df[df['name'] == each].iloc[0]['value']

            # Retrieve reporting unit if it exists
            unit = df[df['name'] == each].iloc[0]['unit']
            break

        except:
            pass

    return ({"primary_assets": primary_assets, "unit": unit})


def summarise_set(doc, variable_names):
    """
    Takes a document (dict) after extraction, and tries to extract
    summary variables relating to the financial state of the enterprise
    by returning all those named that exist.  Returns dict.

    Keyword arguments:
    doc -- an extracted document dict, with "elements" entry as created
           by the 'scrape_clean_elements' functions.
    variable_names - variables to find and return if they exist.
    """
    results = {}

    # Convert elements to pandas df
    df = pd.DataFrame(doc['elements'])

    # Subset to most recent (latest dated)
    df = df[df['date'] == doc['doc_balancesheetdate']]

    # Find all the variables of interest should they exist
    for each in variable_names:
        try:
            results[each] = df[df['name'] == each].iloc[0]['value']

        except:
            pass

    # Send the variables back to be appended
    return (results)


def scrape_elements(soup, filepath):
    """
    Parses an XBRL (xml) company accounts file
    for all labelled content and extracts the
    content (and metadata, eg; unitref) of each
    element found to a dictionary 

    params: filepath (str)
    output: list of dicts
    """

    # Try multiple methods of retrieving data, I think only the first is
    # now needed though.  The rest will be removed after testing this
    # but should not affect execution speed.
    try:
        element_set = soup.find_all()
        elements = parse_elements(element_set, soup)
        if len(elements) <= 5:
            raise Exception("Elements should be gte 5, was {}".format(len(elements)))
        
    except:
        #if fails parsing create dummy entry elements so entry still exists in dictonary
        elements = [{'name': None, 'value': None, 'unit': None, 'date': None}]
        pass

    return (elements)


def flatten_data(doc):
    """
    Takes the data returned by process account, with its tree-like
    structure and reorganises it into a long-thin format table structure
    suitable for SQL applications.
    """

    # Need to drop components later, so need copy in function
    doc2 = doc.copy()
    doc_df = pd.DataFrame()

    # Pandas should create series, then columns, from dicts when called
    # like this
    
    if not isinstance(doc2['elements'], int):
        for element in doc2['elements']:
            doc_df = doc_df.append(element, ignore_index=True)

    # Dump the "elements" entry in the doc dict
    doc2.pop("elements")

    # Create uniform columns for all other properties
    for key in doc2:
        doc_df[key] = doc2[key]
    return (doc_df)

def amp_correction(filepath):
    """
    Looks for & symbol in html file and uses regex to strip out any instances that are not strictly amp;
    
    Named arguments:
    filepath -- complete filepath (string) from drive root
    """
    #import re 
    
    #open file as txt
    #file = open(filepath, 'r')

    #corrected_file = ""
    #for line in file:
        #line = line.strip()
        #print(line)
        
    #return file 
        #too generalised!
        #determine if any lines need correcting
        #if re.search(r'&(?!amp;)',line) is not None:
            #print(line)
            #print message to show correction
            #print('file {} is being corrected for a misplaced & sign, may cause some strange parsing'.format(file))
            #replace with acceptable syntax
            #line = re.sub(r'&(?!amp;)',r'&amp;',line)
    #append subbed lines back to file
    #corrected_file += line
    #return corrected file
    
    #return(corrected_file)


def process_account(filepath):
    """
    Scrape all of the relevant information from
    an iXBRL (html) file, upload the elements
    and some metadata to a mongodb.

    Named arguments:
    filepath -- complete filepath (string) from drive root
    """
    doc = {}

    # Some metadata, doc name, upload date/time, archive file it came from
    doc['doc_name'] = filepath.split("/")[-1]
    doc['doc_type'] = filepath.split(".")[-1].lower()
    doc['doc_upload_date'] = str(datetime.now())
    doc['arc_name'] = filepath.split("/")[-2]
    doc['parsed'] = True

    # Complicated ones
    sheet_date = filepath.split("/")[-1].split(".")[0].split("_")[-1]
    doc['doc_balancesheetdate'] = datetime.strptime(sheet_date, "%Y%m%d").date().isoformat()

    doc['doc_companieshouseregisterednumber'] = filepath.split("/")[-1].split(".")[0].split("_")[-2]

    try:
        soup = BS(open(filepath, "rb"), "html.parser")
        
    except:
        print("Failed to open: " + filepath)
        return (1)

    # Get metadata about the accounting standard used
    try:
        doc['doc_standard_type'], doc['doc_standard_date'], doc['doc_standard_link'] = retrieve_accounting_standard(soup)
        doc['parsed'] = True
    except:
        doc['doc_standard_type'], doc['doc_standard_date'], doc['doc_standard_link'] = (0, 0, 0)
        doc['parsed'] = False

    # Fetch all the marked elements of the document
    try:
        doc['elements'] = scrape_elements(soup, filepath)
    except Exception as e:
        doc['parsed'] = False
        doc['Error'] = e

    try:
        return (doc)
    except Exception as e:
        return (e)


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<a id='section1'></a>
<div style="
    width: 100%;
    height: 100px;
    border:none;
    background:linear-gradient(-45deg, #d84227, #ce3587);
    text-align: center;
    position: relative;
">
  <span style="
      font-size: 40px;
      padding: 0 10px;
      color: white;
      margin: 0;
      position: absolute;
      top: 50%;
      left: 50%;
      -ms-transform: translate(-50%, -50%);
      transform: translate(-50%, -50%);
      font-weight: bold;
  ">
    EXTRACTION <!--Padding is optional-->
  </span>
</div><a id='section1'></a>

In [2]:
import os
import numpy as np
import pandas as pd
import importlib
import time
import sys

def get_filepaths(directory):

    """ Helper function - 
    Get all of the filenames in a directory that
    end in htm* or xml.
    Under the assumption that all files within
    the folder are financial records. """

    files = [directory + "/" + filename
                for filename in os.listdir(directory)
                    if (("htm" in filename.lower()) or ("xml" in filename.lower()))]
    
    month_and_year = ('').join(directory.split('-')[-1:])
    month, year = month_and_year[:-4], month_and_year[-4:]
    
    return files, month, year

def progressBar(name, value, endvalue, bar_length = 50, width = 20):
        percent = float(value) / endvalue
        arrow = '-' * int(round(percent*bar_length) - 1) + '>'
        spaces = ' ' * (bar_length - len(arrow))
        sys.stdout.write(
            "\r{0: <{1}} : [{2}]{3}%   ({4} / {5})".format(
                name,
                width,
                arrow + spaces,
                int(round(percent*100)),
                value,
                endvalue
            )
        )
        sys.stdout.flush()
        if value == endvalue:     
             sys.stdout.write('\n\n')
                
def retrieve_list_of_tags(dataframe, column, output_folder):
    """
    Save dataframe containing all unique tags to txt format in specified directory.
    
    Arguements:
        dataframe:     tabular data
        column:        location of xbrl tags
        output_folder: user specified file location
    Returns:
        None
    Raises:
        None
    """
    list_of_tags = results['name'].tolist()
    list_of_tags_unique = list(set(list_of_tags))

    print(
        "Number of tags in total: {}\nOf which are unique: {}".format(len(list_of_tags), len(list_of_tags_unique))
    )
    
    with open(output_folder + "/" + folder_year + "-" + folder_month + "_list_of_tags.txt", 'w') as f:
        for item in list_of_tags_unique:
            f.write("%s\n" % item)
            
def get_tag_counts(dataframe, column, output_folder):
    """
    Save dataframe containing all unique tags to txt format in specified directory.
    
    Arguements:
        dataframe:     tabular data
        column:        location of xbrl tags
        output_folder: user specified file location
    Returns:
        None
    Raises:
        None
    """
    cache = dataframe
    cache["count"] = cache.groupby(by = column)[column].transform("count")
    cache.sort_values("count", inplace = True, ascending = False)
    cache.drop_duplicates(subset = [column, "count"], keep = "first", inplace = True)
    cache = cache[[column, "count"]]
    
    print(cache.shape)
    
    cache.to_csv(
        output_folder + "/" + folder_year + "-" + folder_month + "_unique_tag_frequencies.csv",
        header = None,
        index = True,
        sep = "\t",
        mode = "a"
    )
    
def build_month_table(list_of_files):
    """
    """
    process_start = time.time()
    
    # Empty table awaiting results
    results = pd.DataFrame()

    COUNT = 0

    # For every file
    #for file in files:
    for file in files:
        COUNT += 1
        
        # Read the file
        doc = process_account(file)

        # tabulate the results
        doc_df = flatten_data(doc)

        # append to table
        results = results.append(doc_df)

        progressBar("XBRL Accounts Parsed", COUNT, len(files), bar_length = 50, width = 20)
    
    print("Average time to process an XBRL file: \x1b[31m{:0f}\x1b[0m".format((time.time() - process_start) / 60, 2), "seconds")
    
    return results
                
def output_xbrl_month(dataframe, output_folder, file_type = "csv"):
    """
    Save dataframe to csv format in specified directory, with particular naming scheme "YYYY-MM_xbrl_data.csv".

    Arguments:
        dataframe:     tabular data
        output_folder: user specified file destination
    Returns:
        None
    Raises:
        None
    """
    if file_type == "csv":
        dataframe.to_csv(
            output_folder
                + "/"
                + folder_year
                + "-"
                + folder_month
                + "_xbrl_data.csv",
            index = False,
            header = True
        )
    else:
        print("I need a CSV for now...")

In [3]:
# Get all the filenames from the example folder
files, folder_month, folder_year = get_filepaths("/shares/data/20200519_companies_house_accounts/xbrl_unpacked_data/Accounts_monthly_Data-June2011")

print(len(files))

# Here you can splice/truncate the number of files you want to process for testing
#files = files[791:793]

61697


In [4]:
print(folder_month, folder_year)

June 2011


### Use these commands to inspect portions of the XML data:
```python
doc = process_account(files[0:])

# display for fun
doc

doc['elements']

# Loop through the document, retrieving any element with a matching name
for element in doc['elements']:
    if element['name'] == 'balancesheetdate':
        print(element)
        
# Extract the all the data to long-thin table format for use with SQL
# Note, tables from docs should be appendable to one another to create
# tables of all data
flatten_data(doc).head(15)
```

In [5]:
# Finally, build a table of all variables from all example (digital) documents
# This can take a while
    
results = build_month_table(files)

print(results.shape)

print(results.doc_name.drop_duplicates().count(),len(files))
results.head(10)

XBRL Accounts Parsed : [------------------------------------------------->]100%   (2 / 2)

Average time to process an XBRL file: 0.012294 seconds
(70, 14)
2 2


,date,name,unit,value,doc_name,doc_type,doc_upload_date,arc_name,parsed,doc_balancesheetdate,doc_companieshouseregisterednumber,doc_standard_type,doc_standard_date,doc_standard_link
0,None,None,None,None,Prod224_9967_01082116_20101231.xml,xml,2020-08-13 15:05:33.371453,Accounts_monthly_Data-June2011,False,2010-12-31,01082116,0,0,0
0,2010-09-30,companynotdormant,NA,true,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
1,2010-09-30,companieshouseregisterednumber,NA,01082277,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
2,2010-09-30,entitycurrentlegalname,NA,MANYHILL LIMITED,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
3,2010-09-30,balancesheetdate,NA,2010-09-30,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
4,2010-09-30,tangiblefixedassets,iso4217:GBP,17078,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
5,2009-09-30,tangiblefixedassets,iso4217:GBP,14311,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
6,2010-09-30,tangiblefixedassetscostorvaluation,iso4217:GBP,82421,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
7,2009-09-30,tangiblefixedassetscostorvaluation,iso4217:GBP,85721,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...
8,2010-09-30,tangiblefixedassetsadditions,iso4217:GBP,10500,Prod224_9967_01082277_20100930.xml,xml,2020-08-13 15:05:33.413685,Accounts_monthly_Data-June2011,True,2010-09-30,01082277,uk-gaap-ae,2008-04-06,http://www.companieshouse.gov.uk/ef/xbrl/uk/fr...


In [6]:
# Hi Rob,
# 
# Just checked several example batch files against the parser's outputs and QA checks,
# the shape in the above cell is always exactly 1 less than the number of rows in the
# outputted csv - as expected with the header row.
# 
# Don't seem to get any error, and I've updated the methods in this notebook further.
# 
# Let me know how you get on with this version!
# 
# Elliott.

output_xbrl_month(results, "/shares/data/20200519_companies_house_accounts/xbrl_parsed_data")

In [7]:
# Find list of all unqiue tags in dataset

list_of_tags = results["name"].tolist()
list_of_tags_unique = list(set(list_of_tags))

print("Longest tag: ", len(max(list_of_tags_unique, key = len)))

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Output all unqiue tags to a txt file

retrieve_list_of_tags(
    results,
    "name",
    "/shares/data/20200519_companies_house_accounts/logs"
)

In [ ]:
# Output all unqiue tags and their relative frequencies to a txt file

get_tag_counts(
    results,
    "name",
    "/shares/data/20200519_companies_house_accounts/logs"
)

In [ ]:
# print(results.shape)